In [17]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch.autograd import Variable
from torchviz import make_dot

In [18]:
class FNN(nn.Module):
    def __init__(self, input_size):
        super(FNN, self).__init__()
        self.l1 = nn.Linear(input_size*2, 5)
        self.relu = nn.ReLU()
        self.l12 = nn.Linear(5, 10)
        self.l2 = nn.Linear(10, 1)

    def forward(self, x):
        out = self.relu(self.l1(x))
        out = self.relu(self.l12(out))
        out = self.relu(self.l2(out))
        return out

In [19]:
data = torch.rand(10,4,2)
labs = torch.rand(10)
learning_rate = 0.001

In [20]:
model = FNN(len(data[0]))
loss_func = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#Train:
sample = data.reshape(len(data), -1)
labels = labs.view(labs.shape[0], 1)

# forward
output = model(sample)
loss = loss_func(output, labels)

# backward
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [21]:
make_dot(output, params=dict(list(model.named_parameters()))).render("simple_fnn2", format="png")

'simple_fnn2.png'